In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama
import pandas as pd

In [2]:
# Step 1: Load the data from Excel
loader = CSVLoader(file_path='unit_ass.csv', csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['','Unit','Topic','Contents','Teaching Hours','Course Outcomes','Similarity Score','Verbs','Assessments']
})

documents = loader.load()


In [3]:
documents

[Document(metadata={'source': 'unit_ass.csv', 'row': 0}, page_content=': \nUnit: Unit\nTopic: Topic\nContents: Contents\nTeaching Hours: Teaching Hours\nCourse Outcomes: Course Outcomes\nSimilarity Score: Similarity Score\nVerbs: Verbs\nAssessments: Assessments'),
 Document(metadata={'source': 'unit_ass.csv', 'row': 1}, page_content=': 0\nUnit: Unit 1\nTopic: introduction to digital image and video processing\nContents: Digital image representation, Sampling and Quantization, Types of Images, Basic Relations between Pixels - Neighbors, Connectivity, Distance Measures between pixels, Introduction to Digital Video, Sampled Video, Video Transmission. Gray-Level Processing: Image Histogram, Linear and Non-linear point operations on Images, Image Thresholding, Region labelling, Binary Image Morphology.\nTeaching Hours: 12\nCourse Outcomes: apply the image and video analysis approaches to solve real world problems\nSimilarity Score: 0.23259731727346955\nVerbs: applysolve\nAssessments: ["[\'a

In [4]:
# Step 2: Extract structured data from loaded documents
# data = []
# for doc in documents:
#     # Extract content from each document
#     content = doc.page_content.strip()
    
#     # Split the content by newline characters to separate columns
#     content_lines = content.split('\n')
    
#     # Ensure there are enough parts in the split content
#     if len(content_lines) >= 8:
#         entry = {
#             "Unit": content_lines[0].strip(),  
#             "Topic":content_lines[2].strip(),       # First line: Unit
#             "Contents": content_lines[3].strip(),       # Third line: Contents
#             "Assessments": content_lines[8].strip()     # Eighth line: Assessments
#         }
#         data.append(entry)

In [5]:


# Step 2: Extract structured data from loaded documents
data = []
for doc in documents:
    # Extract content from each document
    content = doc.page_content.strip()
    
    # Split the content by newline characters to separate columns
    content_lines = content.split('\n')
    
    # Ensure there are enough parts in the split content
    if len(content_lines) >= 9:
        entry = {
            "Unit": content_lines[0].strip(),          # First line: Unit
            "Topic": content_lines[2].strip(),         # Second line: Topic
            "Contents": content_lines[3].strip(),      # Third line: Contents
            "Assessments": set(content_lines[8].strip().split(';'))  # Ninth line: Assessments (handling repeated assessments)
        }
        # Join the assessments back into a single string (if they were split into a set)
        entry["Assessments"] = ', '.join(entry["Assessments"])
        
        data.append(entry)




In [6]:
data

[{'Unit': ':',
  'Topic': 'Topic: Topic',
  'Contents': 'Contents: Contents',
  'Assessments': 'Assessments: Assessments'},
 {'Unit': ': 0',
  'Topic': 'Topic: introduction to digital image and video processing',
  'Contents': 'Contents: Digital image representation, Sampling and Quantization, Types of Images, Basic Relations between Pixels - Neighbors, Connectivity, Distance Measures between pixels, Introduction to Digital Video, Sampled Video, Video Transmission. Gray-Level Processing: Image Histogram, Linear and Non-linear point operations on Images, Image Thresholding, Region labelling, Binary Image Morphology.',
  'Assessments': 'Assessments: ["[\'activities such as case studies, critiques, labs, papers, projects, debates, or concept maps that require students to: discriminate or select relevant and irrelevant parts â€¢ determine how elements function together â€¢ determine bias, values, or underlying intent in presented material â€¢\', \'activities such as papers, exams, problem 

In [9]:

# Step 3: Define a prompt template
prompt_template = PromptTemplate(
    input_variables=["unit", "contents", "assessments"],
    template=(
        """
You are an AI specializing in educational assessments, designed to work through tasks step by step to generate targeted and comprehensive assessments. Below is information about a unit:
Unit: {unit}
Contents: {contents}
Vague Assessments: {assessments}

Task:

Identify Assessment Types: Analyze the provided vague assessments and identify a broad range of specific assessment types, such as 'essay,' 'MCQ,' 'fill in the blank,' 'case study,' 'diagram labeling,' 'problem-solving,' 'coding exercises,' or others.
Link to Content: Extract the key topics and concepts from the unit's content that align with the identified assessment types.
Generate Tailored Assessments: For each assessment type, create detailed and actionable assessments based on the unit's content. Examples include:
Essay: If the content includes 'word embeddings,' the assessment could be: "Write a detailed essay explaining the concept of word embeddings and their role in natural language processing."
Fill in the Blank: For the topic 'vector representations,' the assessment could be: "_____ is the process of representing words as vectors in a high-dimensional space."
Case Study: If the content covers 'applications of word embeddings,' the assessment could be: "Analyze a case study where word embeddings were used to improve search engine results."
Diagram Labeling: If the content involves 'neural network structures,' the assessment could be: "Label the components of the diagram showing the architecture of a word embedding model."
Problem-Solving: For a topic on 'cosine similarity,' the assessment could be: "Calculate the cosine similarity between the following two word vectors."
Deliverable: Provide diverse, specific, and measurable assessments tailored to the unit, ensuring alignment with the content and coverage of a wide range of assessment types.
"""
    )
)


# Step 4: Initialize the LLM and create a chain
llama = Ollama(model="llama3.2")
llm_chain = LLMChain(llm=llama, prompt=prompt_template)

# Step 5: Generate specific assessments
specific_assessments = []
for entry in data:
    response = llm_chain.run({
        "unit": entry["Unit"],
        "contents": entry["Contents"],
        "assessments": entry["Assessments"]
    })
    specific_assessments.append({"Unit": entry["Unit"], "Specific Assessments": response})

# Step 6: Save the results to a CSV file
df = pd.DataFrame(specific_assessments)
output_file = "specific_assessments.csv"
df.to_csv(output_file, index=False)

print(f"Specific assessments have been saved to {output_file}")



C:\Users\rajan\AppData\Local\Temp\ipykernel_5656\453503630.py:28: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llama = Ollama(model="llama3.2")
C:\Users\rajan\AppData\Local\Temp\ipykernel_5656\453503630.py:29: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llama, prompt=prompt_template)
C:\Users\rajan\AppData\Local\Temp\ipykernel_5656\453503630.py:34: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run({


Specific assessments have been saved to specific_assessments.csv
